In [13]:
from sympy import Symbol, Function, Matrix, zeros
import sympy
import numpy as np
import matplotlib.pyplot as plt

We consider a quartic well, specified by restricting a 4th degree polynomial by

1. $U(-1) = 0$
2. $U(0) = U_0$
2. $U(1) = \Delta U$
3. $U'(-1) = 0$
4. $U'(1) = 0$

We get

$$
U(x) = A \left( (x - 1)^2 - \frac{1}{4} \Delta U (x - 2) \right) (x + 1)^2
$$

Q matrix:

$$
\begin{aligned}
Q[\mathbf{x}, \delta \mathbf{x}] & = \sum_{i,j}^d \sum_{k,l}^\infty d^i_k d_l^j \int_0^T dt \left[
\sqrt{\lambda_k \lambda_l} \phi_k \phi_l \frac{\partial^2 L}{\partial x_i \partial x_j}
+ \sqrt{\lambda_l} \psi_k \phi_l \frac{\partial^2 L}{\partial \dot{x}_i \partial x_j}
+ \psi_k \psi_l \frac{\partial^2 L}{\partial \dot{x}_i \partial \dot{x}_j}
\right] \\
& = \sum_{i,j}^d \sum_{k,l}^\infty d^i_k d_l^j Q^{ij}_{kl}
\end{aligned}
$$

In [17]:
dim = 1

In [89]:
t = Symbol('t')
T = Symbol('T', nonnegative=True)
A = Symbol('A')
dU = Symbol('dU')
g = Symbol('gamma')
b = Symbol('beta')

x = Symbol('x')
xdot = Symbol('xdot')

coords = [x]
dcoords = [xdot]

U = A * ( (x-1)**2 - dU * (x-2) / 4 ) * (x+1)**2
F = -U.diff(x)
F = Function('F')(x)

L_OM = (b*g/4) * (xdot - F/g)**2 + (1/(2*g)) * F.diff(x)
L_FW = (b*g/4) * (xdot - F/g)**2

F_expr = -U.diff(x)
div_F_expr = F_expr.diff(x)

In [28]:
def get_L_derivs(L):
    L_x_x = zeros(dim,dim)
    L_xd_x = zeros(dim,dim)
    L_xd_xd = zeros(dim,dim)

    for i in range(dim):
        for j in range(dim):
            L_x_x[i,j] = L.diff(coords[i]).diff(coords[j])
            L_x_x[i,j] = L_x_x[i,j].subs(F, F_expr).simplify()

            L_xd_x[i,j] = L.diff(dcoords[i]).diff(coords[j])
            L_xd_x[i,j] = L_xd_x[i,j].subs(F, F_expr).simplify()

            L_xd_xd[i,j] = L.diff(dcoords[i]).diff(dcoords[j])
            L_xd_xd[i,j] = L_xd_xd[i,j].subs(F, F_expr).simplify()
            
    return L_x_x, L_xd_x, L_xd_xd

In [30]:
L_x_x_OM, L_xd_x_OM, L_xd_xd_OM = get_L_derivs(L_OM)
L_x_x_FW, L_xd_x_FW, L_xd_xd_FW = get_L_derivs(L_FW)

In [74]:
def to_pycode(s):
    return sympy.printing.pycode(s)

In [66]:
py_force_exprs = [U, F_expr, div_F_expr]
py_force_exprs = list(map(to_pycode, py_force_exprs))

py_exprs = []

for i in range(dim):
    for j in range(dim):
        py_exprs.append(L_x_x_OM[i,j])
        py_exprs.append(L_xd_x_OM[i,j])
        py_exprs.append(L_xd_xd_OM[i,j])
        
for i in range(dim):
    for j in range(dim):
        py_exprs.append(L_x_x_FW[i,j])
        py_exprs.append(L_xd_x_FW[i,j])
        py_exprs.append(L_xd_xd_FW[i,j])
        
py_exprs = list(map(to_pycode, py_exprs))

A*(x + 1)**2*(-dU*(x - 2)/4 + (x - 1)**2)
-A*(x + 1)**2*(-dU/4 + 2*x - 2) - A*(2*x + 2)*(-dU*(x - 2)/4 + (x - 1)**2)
-2*A*(x + 1)**2 - 2*A*(2*x + 2)*(-dU/4 + 2*x - 2) - 2*A*(-dU*(x - 2)/4 + (x - 1)**2)
(A**2*beta*(6*dU*x - 48*x**2 + 16)**2/4 - 3*A*beta*(dU - 16*x)*(-A*(x + 1)**2*(dU - 8*x + 8) - 2*A*(x + 1)*(dU*(x - 2) - 4*(x - 1)**2) + 4*gamma*xdot)/2 - 96*A)/(8*gamma)
-A*beta*(6*dU*x - 48*x**2 + 16)/8
beta*gamma/2
beta*(A**2*(6*dU*x - 48*x**2 + 16)**2/4 - 3*A*(dU - 16*x)*(-A*(x + 1)**2*(dU - 8*x + 8) - 2*A*(x + 1)*(dU*(x - 2) - 4*(x - 1)**2) + 4*gamma*xdot)/2)/(8*gamma)
-A*beta*(6*dU*x - 48*x**2 + 16)/8
beta*gamma/2


In [57]:
print("""
def Upot(x):
    return %s
def force_func(x):
    f = %s
    div_f = %s
    return f, div_f
def force_vec(x):
    f, div_f = force_func(x)
    return f
""" % tuple(py_force_exprs))


def Upot(x):
    return A*(x + 1)**2*(-1/4*dU*(x - 2) + (x - 1)**2)
def force_func(x):
    f = -A*(x + 1)**2*(-1/4*dU + 2*x - 2) - A*(2*x + 2)*(-1/4*dU*(x - 2) + (x - 1)**2)
    div_f = -2*A*(x + 1)**2 - 2*A*(2*x + 2)*(-1/4*dU + 2*x - 2) - 2*A*(-1/4*dU*(x - 2) + (x - 1)**2)
    return f, div_f
def force_vec(x):
    f, div_f = force_func(x)
    return f



In [86]:
L_derivs_OM_str = """
def L_derivs_OM(x, xdot):
    L_x_x = np.zeros((%s, %s))
    L_xd_x = np.zeros((%s, %s))
    L_xd_xd = np.zeros((%s, %s))
""" % (dim, dim, dim, dim, dim, dim)

for i in range(dim):
    for j in range(dim):
        L_derivs_OM_str += "    L_x_x[%s,%s] = %s\n" % (i,j, to_pycode(L_x_x_OM[i,j]))
        L_derivs_OM_str += "    L_xd_x[%s,%s] = %s\n" % (i,j, to_pycode(L_xd_x_OM[i,j]))
        L_derivs_OM_str += "    L_xd_xd[%s,%s] = %s\n" % (i,j, to_pycode(L_xd_xd_OM[i,j]))

L_derivs_OM_str += "    return L_x_x, L_xd_x, L_xd_xd"

In [87]:
L_derivs_FW_str = """
def L_derivs_FW(x, xdot):
    L_x_x = np.zeros((%s, %s))
    L_xd_x = np.zeros((%s, %s))
    L_xd_xd = np.zeros((%s, %s))
""" % (dim, dim, dim, dim, dim, dim)

for i in range(dim):
    for j in range(dim):
        L_derivs_FW_str += "    L_x_x[%s,%s] = %s\n" % (i,j, to_pycode(L_x_x_FW[i,j]))
        L_derivs_FW_str += "    L_xd_x[%s,%s] = %s\n" % (i,j, to_pycode(L_xd_x_FW[i,j]))
        L_derivs_FW_str += "    L_xd_xd[%s,%s] = %s\n" % (i,j, to_pycode(L_xd_xd_FW[i,j]))
        
L_derivs_FW_str += "    return L_x_x, L_xd_x, L_xd_xd"

In [88]:
print("%s\n%s" % (L_derivs_OM_str, L_derivs_FW_str))


def L_derivs_OM(x, xdot):
    L_x_x = np.zeros((1, 1))
    L_xd_x = np.zeros((1, 1))
    L_xd_xd = np.zeros((1, 1))
    L_x_x[0,0] = (1/8)*((1/4)*A**2*beta*(6*dU*x - 48*x**2 + 16)**2 - 3/2*A*beta*(dU - 16*x)*(-A*(x + 1)**2*(dU - 8*x + 8) - 2*A*(x + 1)*(dU*(x - 2) - 4*(x - 1)**2) + 4*gamma*xdot) - 96*A)/gamma
    L_xd_x[0,0] = -1/8*A*beta*(6*dU*x - 48*x**2 + 16)
    L_xd_xd[0,0] = (1/2)*beta*gamma
    return L_x_x, L_xd_x, L_xd_xd

def L_derivs_FW(x, xdot):
    L_x_x = np.zeros((1, 1))
    L_xd_x = np.zeros((1, 1))
    L_xd_xd = np.zeros((1, 1))
    L_x_x[0,0] = (1/8)*beta*((1/4)*A**2*(6*dU*x - 48*x**2 + 16)**2 - 3/2*A*(dU - 16*x)*(-A*(x + 1)**2*(dU - 8*x + 8) - 2*A*(x + 1)*(dU*(x - 2) - 4*(x - 1)**2) + 4*gamma*xdot))/gamma
    L_xd_x[0,0] = -1/8*A*beta*(6*dU*x - 48*x**2 + 16)
    L_xd_xd[0,0] = (1/2)*beta*gamma
    return L_x_x, L_xd_x, L_xd_xd
